In [1]:
pip install wmi psutil speedtest-cli screeninfo

Note: you may need to restart the kernel to use updated packages.


In [2]:
import platform
import os
import psutil
import speedtest
import wmi
from screeninfo import get_monitors
import subprocess
import uuid

In [3]:
def get_installed_software():
    # Using WMI to get a list of installed software
    wmi_obj = wmi.WMI()
    software_list = [software.Name for software in wmi_obj.Win32_Product()]
    return software_list

def get_internet_speed():
    # Using Speedtest library to measure internet speed
    st = speedtest.Speedtest()
    download_speed = st.download() / 1_000_000  # in Mbps
    upload_speed = st.upload() / 1_000_000  # in Mbps
    return download_speed, upload_speed

def get_screen_resolution():
    # Using the screeninfo library to get screen resolution
    monitors = get_monitors()
    resolutions = [(monitor.width, monitor.height) for monitor in monitors]
    return resolutions

def get_cpu_info():
    system_info = platform.uname()

    # Get CPU model
    cpu_model = system_info.processor

    # Get number of physical cores and threads
    if os.name == 'posix':  # Linux or MacOS
        with open('/proc/cpuinfo', 'r') as f:
            cpu_info = f.read()
            physical_cores = cpu_info.count('processor')
            threads = cpu_info.count('processor\t:')

    elif os.name == 'nt':  # Windows
        import multiprocessing
        physical_cores = multiprocessing.cpu_count()
        threads = os.cpu_count()

    else:
        print("Unsupported operating system")
        return None

    return cpu_model, physical_cores, threads

def get_gpu_model():
    try:
        system_platform = platform.system()

        if system_platform == "Windows":
            # Windows command to get GPU model
            command = 'wmic path win32_videocontroller get caption'
        elif system_platform == "Linux":
            # Linux command to get GPU model
            command = 'lspci | grep -E "VGA|3D" | awk -F ": " \'{print $2}\''
        else:
            print("Unsupported operating system.")
            return None

        # Run the command and capture the output
        result = subprocess.run(command, stdout=subprocess.PIPE, shell=True, text=True)

        # Extract GPU model from the output
        gpu_model = result.stdout.strip()

        return gpu_model

    except Exception as e:
        print(f"Error: {e}")
        return None

def get_ram_size():
    # Using psutil to get RAM size
    ram_size = psutil.virtual_memory().total / (1024 ** 3)  # in GB
    return ram_size

def get_screen_size():
    monitors = get_monitors()

    if monitors:
        # Assuming the first monitor is the primary monitor (laptop screen)
        laptop_screen = monitors[0]

        width_inch = laptop_screen.width_mm / 25.4
        height_inch = laptop_screen.height_mm / 25.4

        # Calculate screen size in inches using diagonal formula
        diagonal_inches = (width_inch**2 + height_inch**2)**0.5

        return diagonal_inches
    else:
        return None

def get_network_info():
    # Using psutil to get network info
    mac_address = ':'.join(['{:02x}'.format((uuid.getnode() >> elements) & 0xff) for elements in range(5, -1, -1)])
    public_ip = speedtest.Speedtest().get_best_server()["host"]
    return mac_address, public_ip

def get_windows_version():
    # Using platform to get Windows version
    return platform.version() 
 

In [4]:
installed_software = get_installed_software()
internet_speed = get_internet_speed()
screen_resolution = get_screen_resolution()

In [5]:
cpu_model, physical_cores, threads = get_cpu_info()
gpu_info = get_gpu_model()

In [6]:
ram_size = get_ram_size()
screen_size = get_screen_size()
mac_address, public_ip = get_network_info()
windows_version = get_windows_version()

In [7]:
# Print the required information
print("1. All Installed Software List:", installed_software, "\n")
print("2. Internet Speed (Mbps):", "Download:", internet_speed[0], "Upload:", internet_speed[1],"\n")
print("3. Screen Resolution:", screen_resolution, "\n")
print("4. CPU Model:", cpu_model)
print("5. Number of Physical Cores: ", physical_cores, " and Threads of CPU:", threads, "\n")
print("6. GPU Model:", gpu_info, "\n")
print("7. RAM Size (GB):", ram_size)
print("8. Screen Size (in inches):", screen_size, "\n")
print("9. Wifi/Ethernet Mac Address:", mac_address)
print("10. Public IP Address:", public_ip, "\n")
print("11. Windows Version:", windows_version)

1. All Installed Software List: ['MySQL Shell 8.0.33', 'MySQL Connector J', 'MySQL Workbench 8.0 CE', 'Windows PC Health Check', 'MySQL Documents 8.0', 'MySQL Examples and Samples 8.0', 'Office 16 Click-to-Run Extensibility Component', 'Office 16 Click-to-Run Localization Component', 'Office 16 Click-to-Run Extensibility Component 64-bit Registration', 'Office 16 Click-to-Run Licensing Component', None, 'Intel(R) Wireless Bluetooth(R)', 'Microsoft Visual C++ 2010  x64 Redistributable - 10.0.40219', 'Microsoft Visual C++ 2008 Redistributable - x86 9.0.30729', 'Intel® Security Assist', 'ISS_Drivers_x64', 'Apple Software Update', 'Microsoft Visual C++ 2013 x86 Minimum Runtime - 12.0.21005', 'Microsoft Visual C++ 2013 x86 Additional Runtime - 12.0.21005', 'Microsoft Visual C++ 2022 X64 Additional Runtime - 14.36.32532', 'Maxx Audio Installer (x64)', 'Microsoft ODBC Driver 17 for SQL Server', 'Teams Machine-Wide Installer', None, 'Epson Printer Connection Checker', 'Microsoft HEVC Media Ext